In [ ]:
# Copyright (c) 2012-2023, NECOTIS
# All rights reserved.
#
# Redistribution and use in source and binary forms, with or without modification,
# are permitted provided that the following conditions are met:
#
#  - Redistributions of source code must retain the above copyright notice,
#    this list of conditions and the following disclaimer.
#  - Redistributions in binary form must reproduce the above copyright notice,
#    this list of conditions and the following disclaimer in the documentation
#    and/or other materials provided with the distribution.
#  - Neither the name of the copyright holder nor the names of its contributors
#    may be used to endorse or promote products derived from this software
#    without specific prior written permission.
#
# THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND
# ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED
# WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED.
# IN NO EVENT SHALL THE COPYRIGHT HOLDER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT,
# INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT
# NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA,
# OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY,
# WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE)
# ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE
# POSSIBILITY OF SUCH DAMAGE.

# Authors: Emmanuel Calvet, Bertrand Reulet,  Jean Rouat (advisors)
# Date: December 1th, 2023
# Organization: Groupe de recherche en Neurosciences Computationnelles et Traitement Intelligent des Signaux (NECOTIS),
# Université de Sherbrooke, Canada


In [1]:
# Import library and models
# NB: you need to import binary_model_design into the python path
import sys
import os
import importlib

# Get the module path and add it to python path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Setting the module 
model_path = os.path.join(module_path, 'model')
MODULE_PATH = f"{model_path}/__init__.py"
MODULE_NAME = "binary_model" # /!\ if you change this, you must also modify the call of the module in all dependant files!
spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)

# Standard library
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

# The model
from tasks.mackeyglass import mackeyglass
from binary_model.binaryModel import binaryModel
from binary_model.utils import (initRandomArchitecture, 
                        initGaussianWeights, dataPath,
                        dataPathPerf, my_corrcoef, rescale)

In [2]:
# The experiment
sim = 'performance_task'
experiment = 'prediction_mackey-glass'

N = 10000 # Reservoir size. Values tested in the paper: [100, 1000, 10000]
duration = 2000
nb_reservoir = 2 # Number of reservoir par value of sigma
nb_trial = 2 # Number of different state initialization of the same reservoir
K = 16 # Connectivity degree of the graph. [1, 2, 3, 4, 5, 6, 8, 16] 
meanWeight = -0.1 # Average weights for the normal distribution: [-0.1, 0.1]
I = 0.2 # Fraction of up neural state (20%) at times t=0

# List of standard deviation sigma (of the normal distribution)
sigmas = [
            0.01, 0.01247, 0.01555, 0.01939, 0.02418, 0.03016, 0.03761, 0.0469, 0.05848,
            0.07293, 0.09094, 0.11341, 0.14142, 0.17636, 0.21992, 0.27425, 0.342, 0.42648,
            0.53183, 0.66321, 0.82704, 1.03134, 1.28611, 1.60381, 2.0, 2.77899, 3.8614,
            5.36539, 7.45519, 10.0, 10.35895, 12.9155, 14.39371, 16.68101, 20.0, 21.54435,
            27.82559, 35.93814, 46.41589, 59.94843, 77.42637, 100.0
        ]

# Values for testing purpose
nb_sigmas = 2
exponents = np.linspace(-2, 2, nb_sigmas)
sigmas = np.power(10, exponents)

print(f'The model will be evaluated on {len(sigmas)} values of sigma(W):')
print('sigma(W) =', sigmas)

The model will be evaluated on 2 values of sigma(W):
sigma(W) = [1.e-02 1.e+02]


In [3]:
mu = meanWeight
# Parameters of the task --------------------------------------
delay = 10 # Prediction of 10 time step in the future
taus = [5, 15] # Parameter controlling the dynamic of Mackey-Glass. Value tested in the paper:[5, 15, 20, 28]

# Parameter of the training -----------------------------
tTrain = 4000 # Time for the training
tConv = 0
tDiscard_train = 1000 # Time discarded for training
tDiscard_test = 500 # Time discarded for testing
transientData = 500 # Time for transient dynamic
optionData = {'spikeCount': False, 'indexActive': False} 
kwargs = {'optionRun':optionData} # kwargs for the experiment

# Parameters of the network -----------------------------
inSize = 1 # Number of input neurons
outSize = 1 # Number of output neurons
I = 0.5 # Proportion of neuron of the reservoir to which the input/output is connected
output_option =  'exclusion' # Option to obtain a non-overlapping set of neurons connected to input and readout
kwargsOut = {'option':output_option,
            'I':I}

# Options of the readout --------------------------------
learning = 'online'
epoch = 4000
optimizer = 'pytorch' 
activation = 'sigmoid'
optim = 'adam'
loss = 'MSE'
scheduler = False
alpha = 0.001

if 'ridge' in optimizer:
    kwargs.update({'beta':1E-08})
elif optimizer == 'pseudoinverse':
    kwargs.update({'activation':activation})
kwargs.update({'optionCost':'mse'})
if optimizer == 'ANN':
    kwargs.update({'alpha':1E-04,
                    'nbiter':epoch,
                    'activation':activation})
elif optimizer == 'pytorch':
    kwargs.update({'optionOptim':{'alpha':alpha,
                        'epoch':epoch,
                        'activation':activation,
                        'optim':optim,
                        'loss':loss,
                        'scheduler':scheduler,
                        }
            })

tag = f"{experiment}_{optimizer}_{activation}_output-{output_option}"

# Dictionary of result ---------------------------------
task = f'delay{delay}'
path = dataPathPerf(sim, experiment, task=task, optimizer=optimizer, activation=activation, nbtrial=nb_trial)
name = path+f'\dicError_{tag}_nbReservoir{nb_reservoir}_N{N}_K{K}_W{mu}_T{nb_trial}'
try:
    dicResult = np.load(name+'.npy', allow_pickle='TRUE').item()
    corr_tau = dicResult['corr']
    infoReservoir = dicResult['infoReservoir']
    print('File loaded:', name)
except:
    print('File not loaded, will create one:', name)
    corr_tau = {tau: {} for tau in taus}
    infoReservoir = {tau: {} for tau in taus}
    dicResult = {'corr':corr_tau,
                    'tau':taus,
                    'nbtrial':nb_trial,
                    'infoReservoir':infoReservoir,
                    'inputs':'all',
                    'optim': kwargs}

SCRIPT: C:\Users\Manu\Documents\GitHub\binary_model_design
File not loaded, will create one: C:\Users\Manu\Documents\GitHub\binary_model_design\results/performance_task/prediction_mackey-glass/delay10/pytorch_sigmoid/nbtrial2\dicError_prediction_mackey-glass_pytorch_sigmoid_output-exclusion_nbReservoir2_N10000_K16_W-0.1_T2


In [4]:
# Create the model
optionData = {'spikeCount':True, 'indexActive':False} # Option for saving data
brain = binaryModel(N, sim=sim, experiment=experiment)
seedConnectivity = 747
initRandomArchitecture(brain, K, seed=seedConnectivity) # Fixed architecture
print(kwargs)

{'optionRun': {'spikeCount': False, 'indexActive': False}, 'optionCost': 'mse', 'optionOptim': {'alpha': 0.001, 'epoch': 4000, 'activation': 'sigmoid', 'optim': 'adam', 'loss': 'MSE', 'scheduler': False}}


In [5]:
## Run the experiment
import glob

for sigma in tqdm(sigmas):
    sigma = np.round(sigma, 5)

    for res_idx in range(nb_reservoir):
        seed=res_idx*100
        print(f'Reservoir #{res_idx}, sigma={sigma}')

        # Initialize weights
        initGaussianWeights(brain, mu, sigma, seed)

        # Input neurons
        optionIn = 'global'
        kwargsIn = {}

        # Input layer initialization
        brain.addInputLayer(inSize=inSize, tag=tag)

        # Readout initialization
        brain.addReadoutLayer(outSize)
        pathBrain = dataPathPerf(sim, experiment, task, optimizer, activation, nb_trial)+f'/sigma{sigma}/seed{seed}'

        # Check network has not been run already
        dataFile = glob.glob(pathBrain+"/metadata_"+tag+f'*N{N}_K{K}*_W{meanWeight}_*.npy')
        if len(dataFile) != 0:
            print('Network already run, next...\n \n')
            continue
    
        # Select each tau of the MG glass equationc
        for i, tau in enumerate(tqdm(taus)):
                
            # Initialize dictionary result 
            if corr_tau.get(tau, None) is None:
                corr_tau.update({tau:{}})
                infoReservoir.update({tau:{}})
            if corr_tau[tau].get(sigma, None) is None:
                corr_tau[tau].update({sigma:[]})
                infoReservoir[tau].update({sigma:[]})

            # Check if this task is already performed
            nbcircuit_run = len(infoReservoir[tau][sigma])
            print(nbcircuit_run, 'already runned circuits')
            if nbcircuit_run >= nb_reservoir:
                print(f'Networks for tau={tau}, sigma={sigma} already run, next...\n \n')
                continue
            elif (nbcircuit_run < nb_reservoir) and (res_idx < nbcircuit_run):
                print(f'Network {res_idx} for tau={tau}, sigma={sigma} already run, next...\n \n')
                continue
            print('------------------------')

            # Initialize inputs and targets for training
            data = rescale(mackeyglass(4*tTrain, d=tau)) # Generate data with MG equation
            data = data[transientData:]
            x_train = data[:tTrain] # Input of the network
            t_train = np.arange(0, tTrain) # time steps of inputs
            y_train = data[tDiscard_train+delay:tTrain+delay] # Target output
            inputs_train = [x_train, t_train] # inputs of the reservoir
                            
            corrTrials = [] # List to save performance (evulated by the correlation)
            # Compute all trials
            for k in range(nb_trial):
                # Display information
                print(f'Reservoir #{res_idx}, trial #{k}')
                print('********************')
                brain.displayInfoNetwork()
                print('********************')    
                print('Mackey Glass, tau =', tau)

                # Input layer initialization 
                brain.setArchitectureIn(None, option=optionIn, I=I, seed=k*100, **kwargsIn)
                brain.setWeightsIn(None, distribution='uniform', seed=k*100)

                # Readout initialization 
                brain.setArchitectureOut(None, distribution='uniform', seed=k*100, **kwargsOut)

                ##################################
                # Training
                ##################################
                print('------------------------')
                # Set input and target training
                u_train = [inputs_train]
                y_train = [y_train]
                
                # TRAIN !
                brain.train(u_trains = u_train,
                            y_targets = y_train,
                            I = I,
                            duration = tTrain,
                            discardTime = tDiscard_train,
                            convergingTime = tConv,
                            option = learning,
                            optimizer = optimizer,
                            tag = tag,
                            **kwargs)
                print('Mean weight out :', np.mean(brain.Wout), np.std(brain.Wout))
                
                ##################################
                # Task on test set
                ##################################
                brain.reset()
                brain.activationFunction(activation)
                print('------------------------')
                print('Task : predict test-set')
                # Predict set-test
                tstart = tTrain
                tend = 2*tTrain
                u_test = data[tstart:tend]
                y_test = data[tDiscard_test+tstart+delay:tend+delay]
                t_test = np.arange(0, len(u_test))
                inputs = [u_test, t_test]
                y_pred, err = brain.read(inputs, duration=len(u_test), I=I,
                                    discardTime=tDiscard_test, convergingTime=tConv,
                                    y_target=y_test, reset=True, tag=tag, **kwargs)
                corrCoeff = my_corrcoef(y_pred, y_test)

                ##################################
                # Performance and storing
                ##################################
                # Error for each tau of MG
                print('Total error on that task :')
                print('Corr(O, T) =', corrCoeff)
                corrTrials.append(corrCoeff)
                print('Done')
                print('------------------------')
            print('==================================')
            corr_tau[tau][sigma].append(corrTrials)
            infoReservoir[tau][sigma].append(brain.metadata)
            print(brain.metadata)

        # Save metadata
        tag_ = tag + f'_seed{seed}'
        brain.saveData(path=pathBrain, tag=tag_)
        brain.saveMetadata(path=pathBrain, tag=tag_)
        brain.reset(deep=True)

        # Save result in a dictionary
        dicResult['corr'].update(corr_tau)
        dicResult['infoReservoir'].update(infoReservoir)
        
        np.save(name+'.npy', dicResult)
    


  0%|                                                                  | 0/2 [00:00<?, ?it/s]

Reservoir #0, sigma=0.01
addReadoutLayer identity None
addReadoutLayer identity None
SCRIPT: C:\Users\Manu\Documents\GitHub\binary_model_design



  0%|                                                                  | 0/2 [00:00<?, ?it/s]

0 already runned circuits
------------------------
Reservoir #0, trial #0
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 0.01
********************
Mackey Glass, tau = 5
Format dimension for: instantiateArchitecture
Dimension after cheking: <class 'list'>, (1, 5000)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
Format dimension for: instantiateArchitecture
Dimension after cheking: <class 'list'>, (1, 5000)
------------------------
---- Train readout ----
--> pytorch regression
---- Read - train ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 1000
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 1000
Training converged after 3999 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension 

C:\Users\Manu\Documents\GitHub\binary_model_design\model\utils.py:1136: RuntimeWarning: invalid value encountered in double_scalars
  return (( x - mean_x ) * ( y - mean_y )).sum() / n / ( std_x * std_y )


...

1 already runned circuits
------------------------
Reservoir #1, trial #0
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 100.0
********************
Mackey Glass, tau = 5
Format dimension for: instantiateArchitecture
Dimension after cheking: <class 'list'>, (1, 5000)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
Format dimension for: instantiateArchitecture
Dimension after cheking: <class 'list'>, (1, 5000)
------------------------
---- Train readout ----
--> pytorch regression
---- Read - train ----
-> Run with inputs ->
Format dimension for: instantiateInputs
Dimension after cheking: <class 'list'>, (1, 4000)
Read, discardTime 1000
---- RUN ! ----
-> Input sent to the network ->
RUN, discardTime 1000
Training converged after 2899 number of epoch
train, W (1, 5000)
train, b (1,)
SetWeightOut A is not None
Format dimension


 50%|█████████████████████████████                             | 1/2 [00:39<00:39, 39.12s/it]

y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.0432882395845943
Total error on that task :
Corr(O, T) = nan
Done
------------------------
{'meanWeights': -0.1, 'stdWeights': 100.0, 'input': 'prediction_mackey-glass_pytorch_sigmoid_output-exclusion', 'activation': 'sigmoid', 'readout': None, 'input_distribution': 'uniform', 'learning': 'online', 'optimizer': 'pytorch', 'convergingTime': 0, 'discardTime': 1000, 'optionOptim': {'alpha': 0.001, 'epoch': 4000, 'scheduler': False}}
1 already runned circuits
------------------------
Reservoir #1, trial #0
********************
---- Network informations ----
Number of neurons N : 10000
Connectivity degree K : 16
Mean weights : -0.1
Std weights : 100.0
********************
Mackey Glass, tau = 15
Format dimension for: instantiateArchitecture
Dimension after cheking: <class 'list'>, (1, 5000)
SetWeightOut A is not None
Format dimension for: instantiateWeights
Dimension after cheking: <class 'list'>, (1, 5000)
Form


100%|█████████████████████████████████████████████████████████| 2/2 [05:59<00:00, 179.51s/it]

y_pred (1, 3500)
YTARGET (1, 3500) <class 'list'>
Average cost of simulation : 0.09292343006822566
Total error on that task :
Corr(O, T) = 0.004868870446586791
Done
------------------------
{'meanWeights': -0.1, 'stdWeights': 100.0, 'input': 'prediction_mackey-glass_pytorch_sigmoid_output-exclusion', 'activation': 'sigmoid', 'readout': None, 'input_distribution': 'uniform', 'learning': 'online', 'optimizer': 'pytorch', 'convergingTime': 0, 'discardTime': 1000, 'optionOptim': {'alpha': 0.001, 'epoch': 4000, 'scheduler': False}}
SAVE C:\Users\Manu\Documents\GitHub\binary_model_design\results/performance_task/prediction_mackey-glass/delay10/pytorch_sigmoid/nbtrial2/sigma100.0/seed100


In [6]:
# Format all results in a dataframe
import pandas as pd
from binary_model.utils import save_df, add_balance

def make_csv_corr(N, mu, K, nbtrial, nbReservoir, 
                   delay, taus, output, optimizer, 
                   activation, sim='performance_task',
                   experiment='prediction_mackey-glass'):
    
    # Save all results
    result_dir = dataPathPerf(sim, experiment)
    commonFolder = os.path.join(result_dir, 'allResults/')
    if not os.path.isdir(commonFolder):
        os.makedirs(commonFolder)

    df_name = f'{experiment}_allReservoirs_N{N}_K{K}_W{mu}_T{nbtrial}'
    df_path = commonFolder + df_name

    task = f'delay{delay}'
    path = dataPathPerf(sim, experiment, task, optimizer, activation, nbtrial)
    tag = f"{experiment}_{optimizer}_{activation}_output-{output}"
    name = f'/dicError_{tag}_nbReservoir{nbReservoir}_N{N}_K{K}_W{mu}_T{nbtrial}.npy'

    print(path+name)
    dicResult = np.load(path+name, allow_pickle='TRUE').item()
        
    infoReservoirs = dicResult['infoReservoir']
    corrReservoirs = dicResult['corr']
    for tau in taus:
        corrReservoir_sigma = corrReservoirs[tau]
        infoReservoir_sigma = infoReservoirs[tau]
 
        sigmas = []
        corr_reservoirs = []
        for n, (sigma, corr_list) in enumerate(corrReservoir_sigma.items()):
            infoReservoir_list = infoReservoir_sigma[sigma]
        
            for i, corr in enumerate(corr_list[0:nbReservoir]):
                
                # Load data
                infoReservoir = infoReservoir_list[i]
                sigma = infoReservoir['stdWeights']
                # Update results
                sigmas.append(sigma)
                corr = np.array(corr)
                corr = corr[~np.isnan(corr)]
                corr_reservoirs.append(np.mean(corr))
                
        # Create dictionary
        dic_circuit = {'stdWeights':sigmas,
                       f'<C>, tau={tau}':corr_reservoirs,
                       }
        # Save csv
        df = pd.DataFrame(dic_circuit)
        df.to_csv(df_path)
        add_balance(df, df_name, commonFolder, N=N, K=K, W=mu, T=nbtrial)


    return df

df = make_csv_corr(N, mu, K, nb_trial, nb_reservoir, 
                               delay, taus, output_option, optimizer, 
                               activation, sim=sim,
                               experiment=experiment)

df.head()

SCRIPT: C:\Users\Manu\Documents\GitHub\binary_model_design
SCRIPT: C:\Users\Manu\Documents\GitHub\binary_model_design
C:\Users\Manu\Documents\GitHub\binary_model_design\results/performance_task/prediction_mackey-glass/delay10/pytorch_sigmoid/nbtrial2/dicError_prediction_mackey-glass_pytorch_sigmoid_output-exclusion_nbReservoir2_N10000_K16_W-0.1_T2.npy
SAVE C:\Users\Manu\Documents\GitHub\binary_model_design\results/performance_task/prediction_mackey-glass\allResults/ prediction_mackey-glass_allReservoirs_N10000_K16_W-0.1_T2
C:\Users\Manu\Documents\GitHub\binary_model_design\results/performance_task/prediction_mackey-glass\allResults/prediction_mackey-glass_allReservoirs_N10000_K16_W-0.1_T2_N10000_K16_W-0.1_T2
SAVE C:\Users\Manu\Documents\GitHub\binary_model_design\results/performance_task/prediction_mackey-glass\allResults/ prediction_mackey-glass_allReservoirs_N10000_K16_W-0.1_T2
C:\Users\Manu\Documents\GitHub\binary_model_design\results/performance_task/prediction_mackey-glass\allResu

C:\Users\Manu\Anaconda3\envs\criticality\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\Manu\Anaconda3\envs\criticality\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,stdWeights,"<C>, tau=15",balance
0,0.01,0.000000,-1.000000
1,0.01,0.000000,-1.000000
2,100.00,0.015982,-0.000798
3,100.00,0.001722,-0.000798
